In [75]:
import numpy as np
import ast
import pytz
import math
import itertools
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_selection import r_regression 
np.random.seed(1337)
df = pd.read_csv('top10K-spotify.csv',index_col=0)
df["type"].value_counts()
df=df.drop(columns=["index","city_1","district_1","district_2","city_3","district_3" ])
df = df[df['city_2'].notna()]
df = df[df['country'].notna()]
df.rename(columns = {'artist':'name'}, inplace = True)

df2=pd.read_csv('data_w_spotify.csv',index_col=None)
df2=df2.drop(columns=["isdone","id"])
df2['genres'] = df2['genres'].fillna('')

df3=df.merge(df2,on='name')

df3['genres']= df3['genres'].str.strip('[]').str.replace("'","").str.split('\s*,\s*')

def create_soup(x):
    return ' '.join(x['genres']) + ' '+ x['city_2'] + ' ' + x['country']+' ' + x['gender']+ ' ' + x['type']

df3['soup'] = df3.apply(create_soup, axis=1)
print(df3.shape,df3.columns)

dict_names = dict(zip(df3.name, df3.spotifyid))

df = df.reset_index()
indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()



def get_recommendations(name,sim,reverse):
    print(sim[0:20])
    # print(name,sim)
    indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()
    idx = indices[name]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=reverse)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

     # Get the movie indices
    indices = [i[0] for i in sim_scores]
    scores = [round(i[1]*100, 2) for i in sim_scores]

    # Return the top 10 most similar movies
     
    top_10=df3['name'].iloc[indices]
    spot_top10=[]
    for item in top_10:
        spot_top10.append(dict_names[str(item)])

    artist=dict_names[str(df3['name'].iloc[idx])]
    
    wantedDict = {
        'mainId': artist,
        'top10': spot_top10,
        'scores': scores
    }
    return wantedDict




(1416, 11) Index(['name', 'gender', 'age', 'type', 'country', 'city_2', 'spotifyid',
       'genres', 'popularity', 'followers', 'soup'],
      dtype='object')


In [76]:
def CVectorizer():
    count = CountVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def CosSim(x):
    return (cosine_similarity(x,x),True)

In [77]:
def TfidVectorizer():
    count = TfidfVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def EuDSim(x):
    return (euclidean_distances(x,), False)

In [78]:
#matrix=TfidVectorizer()
matrix=CVectorizer()
sim,reverse = EuDSim(matrix)
#im,reverse = CosSim(matrix)

get_recommendations("Coldplay",sim,reverse)

[[0.         4.79583152 5.         ... 5.83095189 5.29150262 5.65685425]
 [4.79583152 0.         4.47213595 ... 5.19615242 4.35889894 5.19615242]
 [5.         4.47213595 0.         ... 4.12310563 3.60555128 3.60555128]
 ...
 [5.56776436 5.29150262 2.44948974 ... 4.58257569 4.58257569 4.12310563]
 [4.89897949 2.64575131 4.12310563 ... 4.89897949 4.         4.89897949]
 [5.74456265 5.29150262 3.46410162 ... 4.79583152 4.79583152 4.35889894]]


{'mainId': '4gzpq5DPGxSnKTe4SA8HAU',
 'top10': ['4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '0sYpJ0nCC8AlDrZFeAA7ub',
  '0sYpJ0nCC8AlDrZFeAA7ub',
  '46CitWgnWrvF9t70C2p1Me',
  '0TytASn0kOZkCpjYTrXsBt',
  '7oPftvlwr6VrsViSDV7fJY'],
 'scores': [173.21,
  173.21,
  173.21,
  173.21,
  173.21,
  200.0,
  223.61,
  223.61,
  244.95,
  244.95]}

In [79]:
def Random_tester():
    rand= np.random.randint(0,len(df3),299)
    return rand

sample=Random_tester()
df_sample=pd.DataFrame(sample)
df_sample = df_sample.drop_duplicates()
# df_sample.to_csv("random.csv",index=False)

In [83]:
import requests
from time import sleep

apiKey = 'BQBO4yCnDwkL_i0m3h3TyQDkLE3j0le2VxgJdaMvhKyL7A6LZpsiNIMbsKIVSICsHBMD4PaCKQqTLDesOcSs5awkCm_zQA_6QkJP5TKRcl94CpyG-15eookOh89b6MXB1bASAcqe5oE8Y8fSr14-7R2ounycsws7t4lVwsi-QUW7QZATLfLorKQxV9NX2Kk'
BASE_URL = 'https://api.spotify.com/v1/'
headers = {
    'Authorization': 'Bearer {token}'.format(token=apiKey)
}


rand_df=pd.read_csv("random.csv",index_col=None)
# print(rand_df.head())
spotifyResultsDf1 = pd.DataFrame()
spotifyResultsDf2 = pd.DataFrame()
spotifyResultsDf3 = pd.DataFrame()
i=0
for item,row in rand_df.iterrows():
    id=str(df3['spotifyid'].iloc[row["Num"]])
    topTwentySpotArray = []
    r = requests.get(BASE_URL + 'artists/' + id + '/related-artists', headers=headers)
    rData = r.json()
    for entry in rData['artists']:
        spotId =  entry['id']
        topTwentySpotArray.append(spotId)
        
    if(len(topTwentySpotArray) < 20):
        while len(topTwentySpotArray) < 20:
            topTwentySpotArray.append("")

    print(i)
    i+=1
    numberWanted = str(row['Num'])
    if i < 100:
        spotifyResultsDf1[numberWanted] = topTwentySpotArray
    elif i >= 100 and i < 200:
        spotifyResultsDf2[numberWanted] = topTwentySpotArray
    else:
        spotifyResultsDf3[numberWanted] = topTwentySpotArray

    sleep(0.4)


spotifyResultsDf1.to_csv('spotify_results1.csv')
spotifyResultsDf2.to_csv('spotify_results2.csv')
spotifyResultsDf3.to_csv('spotify_results3.csv')



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
